In [22]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [23]:
def get_notes():
    
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: 
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: 
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


In [24]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100

    pitchnames = sorted(set(item for item in notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    print('Network input shape')
    print(network_input.shape)
    network_output = np_utils.to_categorical(network_output)
    print('Network output shape')
    print(network_output.shape)
    return (network_input, network_output)

In [25]:
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        128,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(128))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print(model.summary())

    return model

In [26]:
def train(model, network_input, network_output):
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=25, batch_size=4, callbacks=callbacks_list)

In [27]:
def train_network():
    
    notes = get_notes()

    n_vocab = len(set(notes))
    print('No of unique pitches :', n_vocab)
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [28]:
train_network()

Parsing midi_songs\Alphonso.mid
Parsing midi_songs\Avalon.mid
Parsing midi_songs\Bewitched.mid
Parsing midi_songs\BigSurMoon.mid
Parsing midi_songs\Bluette.mid
Parsing midi_songs\BrandNewDay.mid
Parsing midi_songs\BreezeAndI.mid
Parsing midi_songs\CantTameTheLion.mid
Parsing midi_songs\DarnThatDream.mid
Parsing midi_songs\DeadMansRope.mid
Parsing midi_songs\DyingYoung.mid
Parsing midi_songs\GhostOfaChance.mid
Parsing midi_songs\GoingBackHome.mid
Parsing midi_songs\HurtSoBad.mid
Parsing midi_songs\IHungMyHead.mid
Parsing midi_songs\IllBeAlrightWithoutYou.mid
Parsing midi_songs\IllBeSeeingYou.mid
Parsing midi_songs\ImGettingSentimentalOverYou.mid
Parsing midi_songs\IveHeardThatSongBefore.mid
Parsing midi_songs\JoyofLife.mid
Parsing midi_songs\JustAGame.mid
Parsing midi_songs\KingKongItself.mid
Parsing midi_songs\Love.mid
Parsing midi_songs\LoveIsALosingGame.mid
Parsing midi_songs\LoveStory.mid
Parsing midi_songs\MyCherieAmour.mid
Parsing midi_songs\MyGuitarWantsToKillYourMama.mid
Parsing

KeyboardInterrupt: 